In [79]:
import pandas as pd
import numpy as np
import os


In [81]:
project_root = r"C:\Users\Diya\OneDrive\Desktop\quant-trading-system"
raw_path = os.path.join(project_root, "data", "raw", "nifty_spot_5min_raw.csv")

df = pd.read_csv(raw_path, parse_dates=["Date"])
df.head()


,Date,open,high,low,close,volume
0,NaT,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
1,2025-10-27 03:45:00+00:00,25851.94921875,25877.599609375,25828.099609375,25861.400390625,0
2,2025-10-27 03:50:00+00:00,25862.849609375,25882.900390625,25862.849609375,25880.94921875,0
3,2025-10-27 03:55:00+00:00,25880.099609375,25891.0,25867.25,25889.05078125,0
4,2025-10-27 04:00:00+00:00,25889.150390625,25899.25,25880.94921875,25896.55078125,0


In [83]:
for col in ["open", "high", "low", "close", "volume"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [85]:
df = df.sort_values("Date")
df = df.ffill().bfill()


In [73]:
df_clean = df.copy()

def remove_outliers_iqr(data, cols):
    for col in cols:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        data = data[(data[col] >= lower) & (data[col] <= upper)]
    return data

df_clean = remove_outliers_iqr(df_clean, ["open", "high", "low", "close", "volume"])

print("Rows after outlier removal:", len(df_clean))


Rows after outlier removal: 4229


In [101]:
# Convert index back to Date column (if Date is missing)
if "Date" not in df_clean.columns:
    df_clean = df_clean.reset_index()

print(df_clean.columns)


Index(['Date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


In [105]:

# STEP 1: Ensure Date is datetime
df_clean["Date"] = pd.to_datetime(df_clean["Date"], errors="coerce")
df_clean = df_clean.dropna(subset=["Date"])
# STEP 2: Remove duplicate timestamps
df_clean = df_clean.drop_duplicates(subset=["Date"], keep="first")

# STEP 3: Set Date as index
df_clean = df_clean.set_index("Date")

# STEP 4: Create full 5-min index and reindex
full_index = pd.date_range(
    start=df_clean.index.min(), 
    end=df_clean.index.max(), 
    freq="5min", 
    tz="UTC")

df_clean = df_clean.reindex(full_index)

df_clean = df_clean.ffill().bfill()

df_clean = df_clean.reset_index().rename(columns={"index": "Date"})

# Final check
print("Rows after cleaning:", len(df_clean))
print(df_clean.head())
print(df_clean.tail())


Rows after cleaning: 23403
                       Date          open          high           low  \
0 2025-10-27 03:45:00+00:00  25851.949219  25877.599609  25828.099609   
1 2025-10-27 03:50:00+00:00  25862.849609  25882.900391  25862.849609   
2 2025-10-27 03:55:00+00:00  25880.099609  25891.000000  25867.250000   
3 2025-10-27 04:00:00+00:00  25889.150391  25899.250000  25880.949219   
4 2025-10-27 04:05:00+00:00  25897.050781  25927.000000  25895.949219   

          close  volume  
0  25861.400391     0.0  
1  25880.949219     0.0  
2  25889.050781     0.0  
3  25896.550781     0.0  
4  25921.500000     0.0  
                           Date          open          high           low  \
23398 2026-01-16 09:35:00+00:00  25697.300781  25699.349609  25687.050781   
23399 2026-01-16 09:40:00+00:00  25694.199219  25695.150391  25682.800781   
23400 2026-01-16 09:45:00+00:00  25682.449219  25698.599609  25681.599609   
23401 2026-01-16 09:50:00+00:00  25694.050781  25705.699219  25694.050

In [107]:
# Convert timezone from UTC to IST
df_clean["Date"] = df_clean["Date"].dt.tz_convert("Asia/Kolkata")

print(df_clean.head())


                       Date          open          high           low  \
0 2025-10-27 09:15:00+05:30  25851.949219  25877.599609  25828.099609   
1 2025-10-27 09:20:00+05:30  25862.849609  25882.900391  25862.849609   
2 2025-10-27 09:25:00+05:30  25880.099609  25891.000000  25867.250000   
3 2025-10-27 09:30:00+05:30  25889.150391  25899.250000  25880.949219   
4 2025-10-27 09:35:00+05:30  25897.050781  25927.000000  25895.949219   

          close  volume  
0  25861.400391     0.0  
1  25880.949219     0.0  
2  25889.050781     0.0  
3  25896.550781     0.0  
4  25921.500000     0.0  


In [109]:
df_clean = df_clean.drop(columns=["volume"])


In [113]:
import os

project_root = r"C:\Users\Diya\OneDrive\Desktop\quant-trading-system"
cleaned_dir = os.path.join(project_root, "data", "cleaned")

# Create folder if not exist
os.makedirs(cleaned_dir, exist_ok=True)

cleaned_path = os.path.join(cleaned_dir, "nifty_spot_5min_cleaned.csv")

# Save df_clean
df_clean.to_csv(cleaned_path, index=False)

print("✅ Saved successfully at:", cleaned_path)


✅ Saved successfully at: C:\Users\Diya\OneDrive\Desktop\quant-trading-system\data\cleaned\nifty_spot_5min_cleaned.csv
